# Download & Processing

The purpose of this notebook is to download, process, and (optionally) export the data.

## Todo list

* The World Bank data should have infrastructure investments by country by year. Import that data. (Ben)
* BRI field is either 1.0 or 0.0, should be true or false (though we can still work with it as is if we want)
* We want to (potentially) do some network analysis. There's a list [here](https://govt.chinadaily.com.cn/topics/state-ownedenterprises/directorofcentralsoes) but it may not be comprehensive.
* What data do we need/like to have to do a network analysis?
* What data do we still want?
* What visualizations do we want to create? We could easily do some visualizations comparing national investment in infrastructure relative to/comparing the investments/construction with Chinese companies.
* The sky is really the limit here. Jupyter can do basically any and all visualizations imaginable.

In [1]:
# Imports here. Some of these are holdovers from a previous project, ignore them.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests, os, io, zipfile
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [3]:
# Download AEI dataset on Chinese contracts
aeidata = requests.get('https://www.aei.org/wp-content/uploads/2022/02/China-Global-Investment-Tracker-2021-Fall-FINAL-2022.2.21-update.xlsx')

In [15]:
# Handle UN data
with io.BytesIO(aeidata.content) as fh:
    with pd.ExcelFile(fh) as aei_all:
        aei_df = pd.read_excel(aei_all, "Dataset 1+2", usecols="C:G, I, K", header=0, skiprows=5, index_col=None).dropna(subset=['BRI'],axis=0)

In [16]:
aei_df

,Investor,Quantity in Millions,Share Size,Transaction Party,Sector,Country,BRI
1191,China National Off-shore Oil (CNOOC),120,0.1,Total and Wintershall,Energy,Argentina,0.0
1192,China National Petroleum Corp. (CNPC),620,0.49,Rosneft,Energy,Russian Federation,1.0
1193,"China National Petroleum Corp. (CNPC), China N...",1400,"10%, 10%","Petrobras, Shell, and Total",Energy,Brazil,0.0
1194,China Petroleum and Chemical (Sinopec),150,NaN,NaN,Energy,Sao Tome,0.0
1195,China Construction Bank,720,0.74,Banco Industrial e Comerical,Finance,Brazil,0.0
...,...,...,...,...,...,...,...
3747,China Communications Construction,270,NaN,NaN,Transport,Philippines,1.0
3748,Shanghai Tunnel Engineering,400,0.7,LT Sambo,Transport,Singapore,1.0
3749,China State Construction Engineering,670,0.08,NaN,Transport,Australia,0.0
3750,Power Construction Corp. (PowerChina),150,NaN,NaN,Energy,Saudi Arabia,1.0


## TODO: everything below here

Most of this is just code that I imported from a previous project, ignore it for now.

In [4]:
# Download World Bank data
wbdata = requests.get('https://databank.worldbank.org/data/download/WDI_excel.zip')

In [5]:
# Handle WB data (a bit harder, we need to unzip the file first)
with io.BytesIO(wbdata.content) as fh:
    with zipfile.ZipFile(fh, mode='r') as zip_wbdata:
        with zip_wbdata.open('WDIEXCEL.xlsx') as wb_excel:
            with pd.ExcelFile(wb_excel) as wb_all:
                # Basically all the data here is in one excel sheet, that's all we need for now
                wb_data = pd.read_excel(wb_all, "Data", usecols="A,C,BD:BM").dropna(thresh=3)

In [6]:
# if NaN use the latest value, drop all the other values from the decade
cols = wb_data.columns.difference(["Country Name", "Indicator Name"])
wb_data["Value"] = wb_data[cols].apply(lambda x: x.dropna().astype(float)[-1], 1)
wb_data = wb_data.drop(cols, axis=1)

In [7]:
wb_names=["Land area", "Electricity access", "Renewable percentage", #"Urban land area", 
          "Nuclear percentage", "Energy use", "Gini index"]

wb_source_names=["Land area (sq. km)", #"Urban land area (sq. km)", 
                 "Access to electricity (% of population)",
                 "Renewable electricity output (% of total electricity output)",
                 "Electricity production from nuclear sources (% of total)",
                 "Energy use (kg of oil equivalent per capita)", "Gini index (World Bank estimate)"]

wb_pivoted = wb_data.pivot(index="Country Name", columns="Indicator Name")
wb_final = pd.DataFrame(wb_pivoted.Value.to_records()).set_index('Country Name')
wb_final.rename(columns=dict(zip(wb_source_names, wb_names)), inplace=True)
wb_final = wb_final[wb_names]

In [80]:
# Merge the data
data = wb_final.merge(un_hdi, left_index=True, right_index=True)
data = data.merge(un_sustainability, left_index=True, right_index=True)
data = data.merge(un_urbanization, left_index=True, right_index=True)
data = data.merge(un_dietary_adequacy, left_index=True, right_index=True)
data = data.dropna()
# There's a few outliers that have unreliable/skewed data. Let's remove those.
data = data.drop(['Malta', 'South Sudan', 'Luxembourg', 'Mauritius'])

In [81]:
# do we need to normalize?
data

,Land area,Electricity access,Renewable percentage,Nuclear percentage,Energy use,Gini index,HDI,UN Sustainability Index,Urbanization,Food sufficiency
Albania,27400.0,100.000000,100.000000,0.000000,808.455840,33.2,0.795,0.83791,61.2,138
Algeria,2381741.0,99.500000,0.322684,0.000000,1327.535510,27.6,0.748,0.90763,73.2,145
Angola,1246700.0,45.670315,53.174928,0.000000,544.609443,51.3,0.581,0.93195,66.2,110
Argentina,2736690.0,100.000000,28.140069,4.166727,2029.922825,42.9,0.845,0.84902,92.0,134
Armenia,28470.0,100.000000,28.336966,31.806452,1015.909886,29.9,0.776,0.84524,63.2,127
...,...,...,...,...,...,...,...,...,...,...
United Kingdom,241930.0,100.000000,24.840050,20.913853,2764.516671,35.1,0.932,0.78067,83.7,137
United States,9147420.0,100.000000,13.228593,19.323778,6803.996607,41.4,0.926,0.83264,82.5,148
Uruguay,175020.0,99.900000,88.558952,0.000000,1386.018099,39.7,0.817,0.85478,95.4,131
Zambia,743390.0,43.000000,96.993824,0.000000,644.976994,57.1,0.584,0.87456,44.1,90


In [112]:
# We further need to do analysis of this data now that we've pared it down. The most important factors I'm using
# is the urbanization normalized to sprawl[ Urbanization % * ("Urban land area(sq. km)"/"Land area (sq. km)")]
# then double normalized.

# Scratch the above. I couldn't get enough recent data for urban land area (in the last 10 years, I could probably
# make it work if I used older data but ehhhhh not great) so I'm gonna work around it by just using urbanization
metrics = pd.DataFrame(data['Urbanization'], columns=['Urbanization'])

# The second factor is, quite simply, a composite factor considering only renewable energy consumption capacity
# ("Access to electricity (% of population)" * low carbon energy sources")
carbondiet = pd.DataFrame(data['Renewable percentage'].add(data['Nuclear percentage']), columns=['Low carbon energy'])
carbondiet = pd.DataFrame(carbondiet['Low carbon energy'].mul(data['Electricity access']), columns=['Low carbon energy'])

metrics = metrics.merge(carbondiet, left_index=True, right_index=True)

# The third factor is simply the UN sustainability index
metrics = metrics.merge(data['UN Sustainability Index'], left_index=True, right_index=True)

# The fourth factor is (HDI/"Energy use (kg of oil equivalent per capita)")
sus_liv = pd.DataFrame(data['HDI'].div(data['Energy use']), columns=['Sustainable livability']) # this is a bit sus

metrics = metrics.merge(sus_liv, left_index=True, right_index=True)

# The fifth factor is the "Gini index (World Bank estimate)"
metrics = metrics.merge(data['Gini index'], left_index=True, right_index=True)

#Finally normalize
metrics = pd.DataFrame(scaler.fit_transform(metrics.values), columns=metrics.columns, index=metrics.index)
metrics

,Urbanization,Low carbon energy,UN Sustainability Index,Sustainable livability,Gini index
Albania,0.548466,1.000000,0.630953,0.343777,0.223958
Algeria,0.695706,0.002975,0.797803,0.188401,0.078125
Angola,0.609816,0.242672,0.856004,0.374660,0.695312
Argentina,0.926380,0.322908,0.657541,0.133941,0.476562
Armenia,0.573006,0.601340,0.648495,0.262553,0.138021
...,...,...,...,...,...
United Kingdom,0.824540,0.457411,0.493969,0.104656,0.273438
United States,0.809816,0.325364,0.618341,0.030269,0.437500
Uruguay,0.968098,0.884677,0.671325,0.198025,0.393229
Zambia,0.338650,0.416936,0.718662,0.314953,0.846354


In [117]:
weighted = metrics
weighted['Urbanization'] *= 15
weighted['Low carbon energy'] *= 35
weighted['UN Sustainability Index'] *= 20
weighted['Sustainable livability'] *= 10
weighted['Gini index'] = 1 - weighted['Gini index']
weighted['Gini index'] *= 10
weighted = pd.DataFrame(scaler.fit_transform(weighted.values), columns=weighted.columns, index=weighted.index) * 100
weighted['Total'] = weighted.sum(axis=1) / 5
weighted

,Urbanization,Low carbon energy,UN Sustainability Index,Sustainable livability,Gini index,Total
Albania,54.846626,100.000000,63.095295,34.377682,77.604167,65.984754
Algeria,69.570552,0.297486,79.780309,18.840081,92.187500,52.135186
Angola,60.981595,24.267243,85.600440,37.466043,30.468750,47.756814
Argentina,92.638037,32.290780,65.754080,13.394113,52.343750,51.284152
Armenia,57.300613,60.133988,64.849471,26.255291,86.197917,58.947456
...,...,...,...,...,...,...
United Kingdom,82.453988,45.741069,49.396927,10.465636,72.656250,52.142774
United States,80.981595,32.536413,61.834107,3.026900,56.250000,46.925803
Uruguay,96.809816,88.467665,67.132532,19.802455,60.677083,66.577910
Zambia,33.865031,41.693552,71.866175,31.495276,15.364583,38.856923


In [121]:
with pd.ExcelWriter("final_output.xlsx") as writer:
    weighted.sort_values(by=['Total'], ascending=False).to_excel(writer, sheet_name='Ordered Results')
    weighted.to_excel(writer, sheet_name='Results')
    metrics.to_excel(writer, sheet_name='Raw Data')
    data.to_excel(writer, sheet_name='Source Data')